In [ ]:
import numpy as np
from skimage import io
from skimage.morphology import medial_axis, skeletonize
from skimage import measure
from skimage import data
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
from PIL import Image

from skimage.measure import label, regionprops
from skimage.morphology import closing, dilation, disk
from skimage.filters import median
from skimage.morphology import binary_opening, disk
import imgviz

def colored_mask(mask, save_path=None):
    lbl_pil = Image.fromarray(mask.astype(np.uint8), mode="P")
    colormap = imgviz.label_colormap()
    # print(colormap, type(colormap), colormap.flatten(), colormap.shape)
    lbl_pil.putpalette(colormap.flatten())
    if save_path is not None:
        lbl_pil.save(save_path)

    return lbl_pil  

def show_2dpoints(pointcluster,s=None,quivers=None,qscale=1):
    # pointcluster should be a list of numpy ndarray
    # This functions would show a list of pint cloud in different colors
    n = len(pointcluster)
    nmax = n
    if quivers is not None:
        nq = len(quivers)
        nmax = max(n,nq)
    
    colors = ['r','g','b','c','m','y','k','tomato','gold']
    if nmax < 10:
        colors = np.array(colors[0:nmax])
    else: 
        colors = np.random.rand(nmax,3)

    fig = plt.figure(num=1)
    ax = fig.add_subplot(1,1,1)

    if s is None:
        s = np.ones(n)*2

    for i in range(n):
        ax.scatter(pointcluster[i][:,0],pointcluster[i][:,1],s=s[i],c=[colors[i]],alpha=0.6)

    if quivers is not None:
        for i in range(nq):
            ax.quiver(quivers[i][:,0],quivers[i][:,1],quivers[i][:,2],quivers[i][:,3],color=[colors[i]],scale=qscale)

    plt.show()


def SVD(points):
    # 二维，三维均适用
    # 二维直线，三维平面
    pts = points.copy()
    # 奇异值分解
    c = np.mean(pts, axis=0)
    A = pts - c # shift the points
    A = A.T #3*n
    u, s, vh = np.linalg.svd(A, full_matrices=False, compute_uv=True) # A=u*s*vh
    normal = u[:,-1]

    # 法向量归一化
    nlen = np.sqrt(np.dot(normal,normal))
    normal = normal / nlen
    # normal 是主方向的方向向量 与PCA最小特征值对应的特征向量是垂直关系
    # u 每一列是一个方向
    # s 是对应的特征值
    # c >>> 点的中心
    # normal >>> 拟合的方向向量
    return u,s,c,normal


def calcu_dis_from_ctrlpts(ctrlpts):
    if ctrlpts.shape[1]==4:
        return np.sqrt(np.sum((ctrlpts[:,0:2]-ctrlpts[:,2:4])**2,axis=1))
    else:
        return np.sqrt(np.sum((ctrlpts[:,[0,2]]-ctrlpts[:,[3,5]])**2,axis=1))


def estimate_normal_for_pos(pos,points,n):
    """
    计算pos处的法向量.
    
    Input：
    ------
    pos: nx2 ndarray 需要计算法向量的位置.
    points: 骨架线的点集
    n: 用到的近邻点的个数
    
    Output：
    ------
    normals: nx2 ndarray 在pos位置处的法向量.
    """
    
    # estimate normal vectors at a given point
    pts = np.copy(points)
    tree = KDTree(pts, leaf_size=2)
    idx = tree.query(pos, k=n, return_distance=False, dualtree=False, breadth_first=False)
    #pts = np.concatenate((np.concatenate((pts[0].reshape(1,-1),pts),axis=0),pts[-1].reshape(1,-1)),axis=0)
    normals = []
    for i in range(0,pos.shape[0]):
        pts_for_normals = pts[idx[i,:],:]
        _,_,_,normal = SVD(pts_for_normals)
        normals.append(normal)
    normals = np.array(normals)
    return normals


def estimate_normals(points,n):
    """
    计算points表示的曲线上的每一个点法向量.
    等同于 estimate_normal_for_pos(points,points,n)

    Input：
    ------
    points: nx2 ndarray 曲线点集.
    n: 用到的近邻点的个数
    
    Output：
    ------
    normals: nx2 ndarray 在points曲线上的每一处的法向量.
    """
    
    pts = np.copy(points)
    tree = KDTree(pts, leaf_size=2)
    idx = tree.query(pts, k=n, return_distance=False, dualtree=False, breadth_first=False)
    #pts = np.concatenate((np.concatenate((pts[0].reshape(1,-1),pts),axis=0),pts[-1].reshape(1,-1)),axis=0)
    normals = []
    for i in range(0,pts.shape[0]):
        pts_for_normals = pts[idx[i,:],:]
        _,_,_,normal = SVD(pts_for_normals)
        normals.append(normal)
    normals = np.array(normals)
    return normals

def get_crack_ctrlpts(centers,normals,bpoints,hband=5,vband=2,est_width=0):
    # main algorithm to obtain crack width
    cpoints = np.copy(centers)
    cnormals = np.copy(normals)

    xmatrix = np.array([[0,1],[-1,0]])
    cnormalsx = np.dot(xmatrix,cnormals.T).T # the normal of x axis
    N = cpoints.shape[0]

    bpixel_and_skeleton = []
    widths = []
    for i in range(N):
        try:
            ny = cnormals[i]
            nx = cnormalsx[i]
            tform = np.array([nx,ny])
            bpoints_loc = np.dot(tform,bpoints.T).T
            cpoints_loc = np.dot(tform,cpoints.T).T
            ci = cpoints_loc[i]

            bl_ind = (bpoints_loc[:,0]-(ci[0]-hband))*(bpoints_loc[:,0]-ci[0])<0
            br_ind = (bpoints_loc[:,0]-ci[0])*(bpoints_loc[:,0]-(ci[0]+hband))<=0
            bl = bpoints_loc[bl_ind] # left points
            br = bpoints_loc[br_ind] # right points

            if est_width>0:
                # 下面的数值 est_width 是预估计的裂缝宽度
                half_est_width = est_width / 2
                blt = bl[(bl[:,1]-(ci[1]+half_est_width))*(bl[:,1]-ci[1])<0]
                blb = bl[(bl[:,1]-(ci[1]-half_est_width))*(bl[:,1]-ci[1])<0]
                brt = br[(br[:,1]-(ci[1]+half_est_width))*(br[:,1]-ci[1])<0]
                brb = br[(br[:,1]-(ci[1]-half_est_width))*(br[:,1]-ci[1])<0]
            else:
                blt = bl[bl[:,1]>np.mean(bl[:,1])]
                if np.ptp(blt[:,1])>vband:
                    blt = blt[blt[:,1]>np.mean(blt[:,1])]

                blb = bl[bl[:,1]<np.mean(bl[:,1])]
                if np.ptp(blb[:,1])>vband:
                    blb = blb[blb[:,1]<np.mean(blb[:,1])]

                brt = br[br[:,1]>np.mean(br[:,1])]
                if np.ptp(brt[:,1])>vband:
                    brt = brt[brt[:,1]>np.mean(brt[:,1])]

                brb = br[br[:,1]<np.mean(br[:,1])]
                if np.ptp(brb[:,1])>vband:
                    brb = brb[brb[:,1]<np.mean(brb[:,1])]

                # blt = bl[bl[:,1]>np.mean(bl[:,1])]
                # if np.ptp(blt[:,1])>vband:
                #    blt = blt[blt[:,1]<ci[1]+50]
                #    #blt = blt[blt[:,1]>np.mean(blt[:,1])] （外侧）
                #    #blt = blt[blt[:,1]<(np.max(blt[:,1])-0.5*np.ptp(blt[:,1]))] （内侧）

                # blb = bl[bl[:,1]<np.mean(bl[:,1])]
                # if np.ptp(blb[:,1])>vband:
                #    blb = blb[blb[:,1]>ci[1]-50]
                #    #blb = blb[blb[:,1]<np.mean(blb[:,1])]
                #    #blb = blb[blb[:,1]>(np.min(blb[:,1])+0.5*np.ptp(blb[:,1]))]

                # brt = br[br[:,1]>np.mean(br[:,1])]
                # if np.ptp(brt[:,1])>vband:
                #    brt = brt[brt[:,1]<ci[1]+50]
                #    #brt = brt[brt[:,1]>np.mean(brt[:,1])]
                #    #brt = brt[brt[:,1]<(np.max(brt[:,1])-0.5*np.ptp(brt[:,1]))]

                # brb = br[br[:,1]<np.mean(br[:,1])]
                # if np.ptp(brb[:,1])>vband:
                #    brb = brb[brb[:,1]>ci[1]-50]
                #    # brb = brb[brb[:,1]<np.mean(brb[:,1])]
                #    # brb = brb[brb[:,1]>(np.min(brb[:,1])+0.5*np.ptp(brb[:,1]))]

            #bh = np.vstack((bl,br))
            #bmax = np.max(bh[:,1])
            #bmin = np.min(bh[:,1])

            #blt = bl[bl[:,1]>bmax-vband] # left top points
            #blb = bl[bl[:,1]<bmin+vband] # left bottom points

            #brt = br[br[:,1]>bmax-vband] # right top points
            #brb = br[br[:,1]<bmin+vband] # right bottom points


            t1 = blt[np.argsort(blt[:,0])[-1]]
            t2 = brt[np.argsort(brt[:,0])[0]]

            b1 = blb[np.argsort(blb[:,0])[-1]]
            b2 = brb[np.argsort(brb[:,0])[0]]


            interp1 = (ci[0]-t1[0])*((t2[1]-t1[1])/(t2[0]-t1[0]))+t1[1]
            interp2 = (ci[0]-b1[0])*((b2[1]-b1[1])/(b2[0]-b1[0]))+b1[1]

            if interp1-ci[1]>0 and interp2-ci[1]<0:
                widths.append([i,interp1-ci[1],interp2-ci[1]])

                interps = np.array([[ci[0],interp1],[ci[0],interp2]])

                interps_rec = np.dot(np.linalg.inv(tform),interps.T).T

                #show_2dpoints([bpointsxl_loc1,bpointsxl_loc2,bpointsxr_loc1,bpointsxr_loc2,np.array([ptsl_1,ptsl_2]),np.array([ptsr_1,ptsr_2]),interps,ci.reshape(1,-1)],s=[1,1,1,1,20,20,20,20])
                interps_rec = interps_rec.reshape(1,-1)[0,:]
                bpixel_and_skeleton.append(interps_rec)
        except:
            print("the %d-th was wrong" % i)
            continue
    bpixel_and_skeleton = np.array(bpixel_and_skeleton)
    widths = np.array(widths)
    # check
    # show_2dpoints([np.array([[ci[0],interp1],[ci[0],interp2]]),np.array([t1,t2,b1,b2]),cpoints_loc,bl,br],[10,20,15,2,2])
    return bpixel_and_skeleton, widths

######################################################
if __name__ == '__main__':
    import cv2
    img_name = '0032'
    # mask_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/tools/boundary_0032_medial_axis_cls_num_all.png'
    path = f'/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb/outputs/labelme_outputs/SegmentationClass/{img_name}.png'
    # path = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/output/projection/CFD_044.jpg'
    # path = '/home/ubunto/图片/2025-06-13_15-51.png'


    # image = io.imread(path, as_gray=True)
    # image_pil = Image.open(path).convert('1')
    # image = np.array(image_pil)

    # 用此方法读取图片，可以自动判断图片的模式，并将其转化为0-1的灰度图
    def judge_image_mode(path):
        img_pil = Image.open(path)
        img = np.array(img_pil)

        # 根据读取的mask的信息，判断后续操作（有的mask数值取值范围为0-1，有的取值范围为0-255）
        if np.max(img) == 255:
            img_pil = Image.open(path).convert('1')
            img = np.array(img_pil)
        else:
            img_pil = Image.open(path)
            img = np.array(img_pil)
        return img_pil, img

    image_pil, image = judge_image_mode(path)
    print('####### np.unique(image) #######', np.unique(image))
    # image中不为1的地方为背景，为1的地方为前景
    # image[image!= True] = 0
    # image[image == True] = 255
    cls_num_list = np.unique(image).tolist()
    cls_num_list.remove(0)
    print('####### cls_num_list #######', cls_num_list)
    
    
    cls_num_mask_dict = {}
    cls_num_skel_dict = {}  # 每个类别的每个骨架
    cls_num_skel_all_dict = {} # 每个类别的骨架相加
    
    cls_num_edge_to_calculate_mid_axis_dict = {} # 每个类别的边界点
    cls_num_skel_calculate_mid_axis_dict = {} # 每个类别的骨架的中轴线
    cls_num_edge_pair_dict = {} # 每个类别的边界点对
    
    for cls_num in cls_num_list:
        print(f"############ cls_num #############: {cls_num}")
        cls_num_mask_dict[cls_num] = []
        cls_num_skel_dict[cls_num] = []
        cls_num_skel_all_dict[cls_num] = []
        
        cls_num_edge_to_calculate_mid_axis_dict[cls_num] = []
        cls_num_skel_calculate_mid_axis_dict[cls_num] = []
        cls_num_edge_pair_dict[cls_num] = []
        
        cls_num_mask = image.copy()
        cls_num_mask[cls_num_mask != cls_num] = 0
        
        num_labels, labels = cv2.connectedComponents(cls_num_mask)
        print(f"连通区域的个数(包含背景)为: {num_labels}")
        print(f"连通区域的标签为: {labels.shape}", np.unique(labels))   
        # 统计mask的个数（减去背景）
        mask_count = num_labels - 1
        
        print(f"mask的个数(含有少量孤立点)为: {mask_count}")
        
        less_than_10_mask_count = []
        # 为每个mask编号（从1开始，因为0通常代表背景）
        ############################# 基于语义分类的实例分类 ################################
        for num_label in range(1, num_labels):
            # single_mask = np.uint8(labels == label) * 255
            # single_mask 分别代表不同label的mask,保持与原label数值一致
            single_mask = np.zeros(cls_num_mask.shape, dtype=np.uint8)
            single_mask[labels == num_label] = num_label
            
            single_mask_pil = colored_mask(single_mask)
            single_mask = np.array(single_mask_pil)
            
            print('np.unique(single_mask):', np.unique(single_mask))
            # 计算single_mask的非零的数量
            non_zero_count = np.count_nonzero(single_mask)
            # print(f"mask={label}的非零的数量为: {non_zero_count}")
            if non_zero_count < 10:
                less_than_10_mask_count.append(num_label)
                mask_count -= 1
                # single_mask[labels == label] = label
                # print('更新后的np.unique(single_mask):', np.unique(single_mask))
                continue 
            else:
                single_mask_new = single_mask.copy()
                cls_num_mask_dict[cls_num].append(single_mask_new)
                print('single_mask_new.shape:', single_mask_new.shape, np.unique(single_mask_new))
            # 这里可以对单个mask进行进一步处理，比如保存图像
            # cv2.imwrite(f'mask_{label}.png', single_mask)
            # cv2.imshow(f'mask_{label}', single_mask)
            # cv2.waitKey(0)
            plt.title(f'mask class is {cls_num}, number id is {num_label}')
            plt.imshow(single_mask_pil)
            # single_mask_pil.save(f'mask_{label}.png')
            plt.show()
            
            # 每个单独的损伤label的骨架提取计算最大宽度
            single_mask_new_copy = single_mask.copy()
            # if type(cls_num) == int:
            single_mask_new_copy[single_mask_new_copy!= num_label] = 0
            single_mask_new_copy[single_mask_new_copy == num_label] = 255
            
            img_pil_new = Image.fromarray(single_mask_new_copy)
    
    # image[image!= 0] = 255
    # image[image == 0] = 0
    
            img_pil_new = Image.fromarray(single_mask_new_copy)
            image_pil = img_pil_new.convert('1')
            image_np = np.array(image_pil)
            print(image_np.shape)
            iw,ih = image_np.shape

            blobs  = np.copy(image_np)
            # blobs[blobs<128] = 0
            # blobs[blobs>128] = 1

            blobs = blobs.astype(np.uint8)
            # Generate the data
            #blobs = data.binary_blobs(200, blob_size_fraction=.2,
                                    #volume_fraction=.35, seed=1)
            # using scikit-image
            ## Compute the medial axis (skeleton) and the distance transform
            #skel, distance = medial_axis(blobs, return_distance=True)
            ## Distance to the background for pixels of the skeleton
            #dist_on_skel = distance * skel

            # Compare with other skeletonization algorithms
            print(blobs)
            skeleton_org = skeletonize(blobs, method='lee')
            # 去除skeleton的多余分支
            labeled = label(skeleton_org)
            regions = regionprops(labeled)
            min_length = 1  # 设定最小分支长度

            mask = np.zeros_like(skeleton_org, dtype=bool)
            for reg in regions:
                if reg.area >= min_length:
                    mask[reg.coords[:, 0], reg.coords[:, 1]] = True
            skeleton = mask.astype(np.uint8)
            # skeleton = skeleton_org
            
            
            # skeleton = skeletonize(blobs)
            print("########### skeleton.shape ###########", skeleton.shape, np.unique(skeleton), skeleton.dtype)
            if skeleton.dtype == np.uint8:
                skeleton = skeleton.astype(np.bool_)

            # skeleton_lee = skeletonize(blobs, method='lee')
            x, y = np.where(skeleton>0)
            centers = np.hstack((x.reshape(-1,1),y.reshape(-1,1)))
            print('centers.shape', centers.shape)

            normals = estimate_normals(centers, 9) # 这个用于估计法向量的KNN

            # search contours of the crack
            contours = measure.find_contours(blobs, 0.8)
            print('len(contours)', len(contours))
            # print(contours[0].shape, contours[1].shape) 
            # bl = contours[0]
            # br = contours[1]

            # bpoints = np.vstack((bl,br))
            bpoints = np.concatenate(contours,axis=0)
            print('bpoints.shape', bpoints.shape)

            #bpixel_and_skeleton, widths = get_crack_ctrlpts(centers,normals,bpoints,hband=2,vband=2)


            bpixel = np.zeros((iw,ih,3),dtype=np.uint8)
            bpoints = bpoints.astype(np.int64)
            bpixel[bpoints[:,0],bpoints[:,1],0] = 255

            skeleton_pixel = np.zeros((iw,ih,3),dtype=np.uint8)
            skeleton_pixel[skeleton,1] = 255

            bpixel_and_skeleton = np.copy(bpixel)
            bpixel_and_skeleton[skeleton,1] = 255


            fig, axes = plt.subplots(2,2, figsize=(30, 15))
            ax = axes.ravel()

            ax[0].imshow(blobs, cmap=plt.cm.gray)
            ax[0].axis('off')


            ax[1].imshow(bpixel_and_skeleton)
            #for contour in contours:
            #    ax[1].plot(contour[:, 1], contour[:, 0], linewidth=2)

            # for n, contour in enumerate(contours):
            #     ax[1].plot(contour[:, 1], contour[:, 0], linewidth=1, color='red')
            #ax[1].set_title('medial_axis')
            ax[1].axis('off')


            ax[2].imshow(bpixel_and_skeleton)
            interps, widths = get_crack_ctrlpts(centers, normals, bpoints, hband=2, vband=2, est_width=100)
            print('interps.shape, widths.shape', interps.shape, widths.shape)
            # 如果interps的x
            # interps_center
            # print(np.random.choice(interps.shape[0], interps.shape[0]), len(np.random.choice(interps.shape[0], interps.shape[0])))
            # print(interps)
            # 计算interps连线和skeleton的交点
            # interps_center = []
            # for i in range(interps.shape[0]):
            #     interps_center.append(np.mean(np.vstack((interps[i,:2],interps[i,2:]),axis=0),axis=0))
            interps_center_x = interps[:, [0,2]].mean(axis=1)
            interps_center_y = interps[:, [1,3]].mean(axis=1)
            interps_center = np.hstack((interps_center_x.reshape(-1,1),interps_center_y.reshape(-1,1)))
            # interps_center_x_ls = []
            # interps_center_y_ls = []
            # for i in range(interps.shape[0]):
            #     interps_center_x_ls.append((interps[i,1]+interps[i,3])/2)
            #     interps_center_y_ls.append((interps[i,0]+interps[i,2])/2)
            # interps_center_x = np.array(interps_center_x_ls)
            # interps_center_y = np.array(interps_center_y_ls)
            interps_center = np.hstack((interps_center_x.reshape(-1,1),interps_center_y.reshape(-1,1)))
            # 每隔5个选取interps_center的点
            inter_num = 5
            interps_center_show = interps_center[::inter_num,:]
            # for i in range(0,interps_center_show.shape[0],5):
            
            print('interps_center.shape', interps_center.shape)
            
            # for i in range(interps.shape[0]):
            #     interps
            
            # interps_show = interps[np.random.choice(interps.shape[0], 120, replace=False),:] # 由于太多，这里随机采样120个测量位置，进行显示
            # 在interps中每隔5个选取interps_show的点
            interps_show = interps[::inter_num,:]
            # for i in range(0,interps_show.shape[0],5):
            #     ax[2].plot([interps_show[i,1],interps_show[i,3]],[interps_show[i,0],interps_show[i,2]],c='c',ls='-',lw=1,marker='o',ms=1,mec='c',mfc='c')
            # interps_show = interps[np.random.choice(interps.shape[0], interps.shape[0]),:] # 由于太多，这里随机采样120个测量位置，进行显示
            for i in range(interps_show.shape[0]):
                ax[2].plot([interps_show[i,1],interps_show[i,3]],[interps_show[i,0],interps_show[i,2]],c='c', ls='-', lw=1, marker='o',ms=1,mec='c',mfc='c')
            
            print('########### interps_show.shape ###########', interps_show.shape)


            
            ax[3].imshow(bpixel_and_skeleton)
            # ax[3]绘制测量点
            ax[3].plot(interps_center_show[:,1],interps_center_show[:,0],c='w',marker='o',ms=3, linestyle='')
            # ax[3].plot(interps_center[:,1],interps_center[:,0],c='c')
            ax[3].set_title('crack width estimation')

            widths_show = widths[::inter_num,:]
            print('widths_show.shape', widths_show.shape)
            print('interps_center_show.shape', interps_center_show.shape)

            ## ================ small window ==================
            #pos = np.array([191, 291]).reshape(1,-1) # input (x,y) where need to calculate crack width
            ## pos = np.array([142, 178]).reshape(1,-1)

            #posn = estimate_normal_for_pos(pos,centers,3)

            #interps, widths2 = get_crack_ctrlpts(pos,posn,bpoints,hband=1.5,vband=2)


            #sx = pos[0,0] - 20
            #sy = pos[0,1] - 20

            #ax[2].imshow(bpixel_and_skeleton)

            #for i in range(interps.shape[0]):
            #    ax[2].plot([interps[i,1],interps[i,3]],[interps[i,0],interps[i,2]],c='c',ls='-',lw=5,marker='o',ms=8,mec='c',mfc='c')

            #ax[2].set_ylim(sx,sx+40)
            #ax[2].set_xlim(sy,sy+40)

            ##ax[2].set_title('skeletonize')
            #ax[2].axis('off')

            #print(interps)
            
            cls_num_skel_all_dict[cls_num].append(skeleton)
            
            cls_num_skel_calculate_mid_axis_dict[cls_num].append(interps_center)
            cls_num_edge_to_calculate_mid_axis_dict[cls_num].append(bpoints)
            cls_num_edge_pair_dict[cls_num].append(interps)
            
            
            fig.tight_layout()
            fig.savefig(f'crack_width_estimation_{img_name}.png', dpi=300)

            plt.show()

import os
save_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/output/dalian_xinghaiwandaqiao_rgb/img_point_width'
os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, 'cls_num_edge_pair_dict.npy'), cls_num_edge_pair_dict)
np.save(os.path.join(save_dir, 'cls_num_edge_to_calculate_mid_axis_dict.npy'), cls_num_edge_to_calculate_mid_axis_dict)
